In [33]:
%matplotlib inline
import loader
import numpy as np
import random
file = '2012-Consolidated-stripped.csv'

In [34]:
reload(loader)
rows = loader.load_raw(file)

In [35]:
diabetes = []
for pid in rows:
    person = rows[pid]
    if 'chronic_diabetes' in person.info and person.info['chronic_diabetes'] is 1:
        diabetes.append(pid)

In [36]:
cared, uncared = [], []
for pid in diabetes:
    person = rows[pid]
    care = 0
    if ('diabetes_a1c' in person.info and person.info['diabetes_a1c']>0):
        care += 1
    if ('diabetes_foot' in person.info and person.info['diabetes_foot'] > 0):
        care += 1
    if ('diabetes_eye' in person.info and person.info['diabetes_eye'] > 0):
        care += 1
    if ('diabetes_cholest' in person.info and person.info['diabetes_cholest'] > 0):
        care += 1
    if (care <= 1):
        uncared.append(pid)
    else:
        cared.append(pid)

In [37]:
print len(cared), len(uncared)

2261 490


In [6]:
def reservoir_sample(iterator, k):
    """
    Basic reservoir sample. Takes a target sample amount
    """
    # fill the reservoir to start
    iterator = iter(iterator)
    result = [next(iterator) for _ in range(k)]
    n = k
    for item in iterator:
        n += 1
        s = random.randint(0, n)
        if s < k:
            result[s] = item
    return result

def get_sample_size(len1, len2, percent):
    return int(min(len1, len2) * percent)

In [7]:
# get samples for cared and uncared groups
ssize = get_sample_size(len(cared), len(uncared), 0.7)
scared = reservoir_sample(cared, ssize)
suncared = reservoir_sample(uncared, ssize)

In [8]:
cared_cost = [rows[pid].info['spending_pay_medicaid']+rows[pid].info['spending_pay_medicare'] for pid in scared]
uncared_cost = [rows[pid].info['spending_pay_medicaid']+rows[pid].info['spending_pay_medicare'] for pid in suncared]

In [9]:
print np.average(uncared_cost), np.average(cared_cost)

6522.20408163 5912.12536443


In [13]:
# age breakdown for cared and uncared groups
cared_young, cared_mid, cared_old = [], [], []
uncared_young, uncared_mid, uncared_old = [], [], []
for pid in cared:
    person = rows[pid]
    if person.info['demo_age']>=65:
        cared_old.append(pid)
    elif person.info['demo_age']>=45:
        cared_mid.append(pid)
    elif person.info['demo_age']>18:
        cared_young.append(pid)

for pid in uncared:
    person = rows[pid]
    if person.info['demo_age']>=65:
        uncared_old.append(pid)
    elif person.info['demo_age']>=45:
        uncared_mid.append(pid)
    elif person.info['demo_age']>18:
        uncared_young.append(pid)

In [14]:
cared_old_cost = [rows[pid].info['spending_pay_medicaid']+rows[pid].info['spending_pay_medicare'] for pid in cared_old]
cared_mid_cost = [rows[pid].info['spending_pay_medicaid']+rows[pid].info['spending_pay_medicare'] for pid in cared_mid]
cared_young_cost = [rows[pid].info['spending_pay_medicaid']+rows[pid].info['spending_pay_medicare'] for pid in cared_young]
uncared_old_cost = [rows[pid].info['spending_pay_medicaid']+rows[pid].info['spending_pay_medicare'] for pid in uncared_old]
uncared_mid_cost = [rows[pid].info['spending_pay_medicaid']+rows[pid].info['spending_pay_medicare'] for pid in uncared_mid]
uncared_young_cost = [rows[pid].info['spending_pay_medicaid']+rows[pid].info['spending_pay_medicare'] for pid in uncared_young]

In [15]:
print '', 'Old', 'Mid', 'Young'
print 'cared', np.average(cared_old_cost), np.average(cared_mid_cost), np.average(cared_young_cost)
print 'uncared', np.average(uncared_old_cost), np.average(uncared_mid_cost), np.average(uncared_young_cost)

 Old Mid Young
cared 8127.99223085 3612.09423077 3071.16352201
uncared 8323.52222222 3760.18905473 1231.84848485
